In [1]:
import numpy as np
import pandas as pd

import sys
sys.path.append('../')

from src.data_utils import *
from src.tda_utils import *
from src.utils import *

import plotly.express as px
import plotly.graph_objects as go

import glob

In [2]:
# list_data_name = ["BTCUSDT_1d", "ETHUSDT_1d", "BNBUSDT_1d", "XRPUSDT_1d", "ADAUSDT_1d", "LTCUSDT_1d"]                   
# list_data_name = ["BTCUSDT_1h", "ETHUSDT_1h", "BNBUSDT_1h", "XRPUSDT_1h", "ADAUSDT_1h", "LTCUSDT_1h"]                   

list_data_name = ["ADAUSDT_1d"]                   
                        
from_date = '2021-12-31 00:00:00' # 2020-12-31 00:00:00
until_date = '2025-07-31 00:00:00'

In [3]:
initial_slice_position = 0
list_slice_size_in_days = [14, 21, 30, 45]
# list_slice_size_in_days = [45]
# list_slice_size_in_days = [14, 21, 30]
num_point_for_1d = (1 * 24) # 1h
column_name = 'processed_log_return_wtmra_0'
save = True

In [4]:
# Parameters for the point cloud creation
dimension = 2
list_time_delay = [3, 7]
# time_delay_in_days = 7
stride = 1

In [5]:
# Create folder for saving results if doesn't exist
def create_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

def erase_files(path): 
    files = glob.glob(path + '*')
    for f in files:
        os.remove(f)

In [6]:
for data_name in list_data_name:
    for slice_size_in_days in list_slice_size_in_days:
        slice_size = slice_size_in_days * num_point_for_1d
        for time_delay in list_time_delay:
            time_delay_in_days = (time_delay * 24)

            # Load the data
            data = pd.read_csv(f'../data/01-output-{data_name}-from-{from_date}-until-{until_date}-log-return.csv', parse_dates=['date'], index_col='date')

            df = data[column_name]

            full_series_with_highlight_path = f'../results/{column_name}_{data_name}_time_delay_{time_delay_in_days}_slice_size_{slice_size}/full_series_with_highlight/'
            sliced_time_series_path = f'../results/{column_name}_{data_name}_time_delay_{time_delay_in_days}_slice_size_{slice_size}/sliced_time_series/'
            point_cloud_path = f'../results/{column_name}_{data_name}_time_delay_{time_delay_in_days}_slice_size_{slice_size}/point_cloud/'
            persistence_diagram_path = f'../results/{column_name}_{data_name}_time_delay_{time_delay_in_days}_slice_size_{slice_size}/persistence_diagram/'
            mapper_graph_path = f'../results/{column_name}_{data_name}_time_delay_{time_delay_in_days}_slice_size_{slice_size}/mapper_graph/'
            mosaic_path = f'../results/{column_name}_{data_name}_time_delay_{time_delay_in_days}_slice_size_{slice_size}/mosaic/'

            features_path = f'../results/{column_name}_{data_name}_time_delay_{time_delay_in_days}_slice_size_{slice_size}/features/'
            features_plot_path = f'../results/{column_name}_{data_name}_time_delay_{time_delay_in_days}_slice_size_{slice_size}/features_plot/'
            mosaic_and_features_path = f'../results/{column_name}_{data_name}_time_delay_{time_delay_in_days}_slice_size_{slice_size}/mosaic_and_features/'
            video_path = f'../results/{column_name}_{data_name}_time_delay_{time_delay_in_days}_slice_size_{slice_size}/video/'


            if save:
                create_folder(full_series_with_highlight_path)
                create_folder(sliced_time_series_path)
                create_folder(point_cloud_path)
                create_folder(persistence_diagram_path)
                create_folder(mapper_graph_path)
                create_folder(mosaic_path)
                create_folder(features_path)
                create_folder(features_plot_path)
                create_folder(mosaic_and_features_path)
                create_folder(video_path)

            if save:
                erase_files(full_series_with_highlight_path)
                erase_files(sliced_time_series_path)
                erase_files(point_cloud_path)
                erase_files(persistence_diagram_path)
                erase_files(mapper_graph_path)
                erase_files(mosaic_path)
                erase_files(features_path)
                erase_files(features_plot_path)
                erase_files(mosaic_and_features_path)
                erase_files(video_path)

            df_features = pd.DataFrame()

            # Create an empty dataframe with these columns: [initial_slice_position, slice_size, start_date, end_date, connected_components_entropy, loops_entropy, voids_entropy, connected_components_amplitude, loops_amplitude, voids_amplitude, connected_components_number_of_points, loops_number_of_points, voids_number_of_points]
            df_features = pd.DataFrame(columns=['initial_slice_position', 'slice_size', 'start_date', 'end_date', 'connected_components_entropy', 'loops_entropy', 'voids_entropy', 'connected_components_amplitude', 'loops_amplitude', 'voids_amplitude', 'connected_components_number_of_points', 'loops_number_of_points', 'voids_number_of_points'])

            for i in range(0, (len(data)-slice_size), num_point_for_1d):

                # fig = generate_plot_full_series_with_highlight(df, initial_slice_position + i, slice_size, save, full_series_with_highlight_path, f'full_series_with_highlight_{generate_numerical_filename(initial_slice_position + i, len(data))}_{slice_size}', data_name)
                time_series = get_sliced_time_series(df, initial_slice_position + i, slice_size)
                # fig = generate_plot_sliced_time_series(df, initial_slice_position + i, slice_size, save, sliced_time_series_path, f'sliced_time_series_{generate_numerical_filename(initial_slice_position + i, len(data))}_{slice_size}')
                # fig.show()
                point_cloud = create_point_cloud(time_series, dimension, time_delay_in_days, stride)
                # fig = generate_plot_point_cloud(point_cloud, initial_slice_position + i, slice_size, save, point_cloud_path, f'point_cloud_{generate_numerical_filename(initial_slice_position + i, len(data))}_{slice_size}')
                # fig.show()
                # fig = generate_plot_persistence_diagram(point_cloud, initial_slice_position + i, slice_size, save, persistence_diagram_path, f'persistence_diagram_{generate_numerical_filename(initial_slice_position + i, len(data))}_{slice_size}')
                # fig.show()
                # fig = plot_persistence_barcode(point_cloud, initial_slice_position + i, slice_size, save, '../results/persistence_barcode/', f'persistence_barcode_{initial_slice_position + i}_{slice_size}')
                # fig.show()
                # fig = plot_persistence_heatmap(point_cloud, initial_slice_position + i, slice_size, save, '../results/persistence_heatmap/', f'persistence_heatmap_{initial_slice_position + i}_{slice_size}')
                # fig.show()
                # fig = plot_persistence_landscape(point_cloud, initial_slice_position + i, slice_size, save, '../results/persistence_landscape/', f'persistence_landscape_{initial_slice_position + i}_{slice_size}')
                # fig.show()
                # fig = generate_plot_mapper_graph(point_cloud, initial_slice_position + i, slice_size, save, mapper_graph_path, f'mapper_graph_{generate_numerical_filename(initial_slice_position + i, len(data))}_{slice_size}')
                # fig.show()

                entropy_features_dict, amplitude_features_dict, number_of_points_features_dict = get_features(point_cloud)

                # Add the features to the dataframe using concat
                df_features = pd.concat([df_features, pd.DataFrame({
                    'initial_slice_position': initial_slice_position + i,
                    'slice_size': slice_size,
                    'start_date': data.index[initial_slice_position + i],
                    'end_date': data.index[initial_slice_position + i + slice_size],
                    'connected_components_entropy': entropy_features_dict['connected_components_entropy'],
                    'loops_entropy': entropy_features_dict['loops_entropy'],
                    'voids_entropy': entropy_features_dict['voids_entropy'],
                    'connected_components_amplitude': amplitude_features_dict['connected_components_amplitude'],
                    'loops_amplitude': amplitude_features_dict['loops_amplitude'],
                    'voids_amplitude': amplitude_features_dict['voids_amplitude'],
                    'connected_components_number_of_points': number_of_points_features_dict['connected_components_number_of_points'],
                    'loops_number_of_points': number_of_points_features_dict['loops_number_of_points'],
                    'voids_number_of_points': number_of_points_features_dict['voids_number_of_points']
                }, index=[0])], ignore_index=True)        

                # full_path_img1 = f'{sliced_time_series_path}/sliced_time_series_{generate_numerical_filename(initial_slice_position + i, len(data))}_{slice_size}.png'
                # full_path_img2 = f'{full_series_with_highlight_path}/full_series_with_highlight_{generate_numerical_filename(initial_slice_position + i, len(data))}_{slice_size}.png'
                # full_path_img3 = f'{point_cloud_path}/point_cloud_{generate_numerical_filename(initial_slice_position + i, len(data))}_{slice_size}.png'
                # full_path_img4 = f'{point_cloud_path}/point_cloud_{generate_numerical_filename(initial_slice_position + i, len(data))}_{slice_size}_fixed_axis.png'
                # full_path_img5 = f'{persistence_diagram_path}/persistence_diagram_{generate_numerical_filename(initial_slice_position + i, len(data))}_{slice_size}.png'
                # full_path_img6 = f'{persistence_diagram_path}/persistence_diagram_{generate_numerical_filename(initial_slice_position + i, len(data))}_{slice_size}_fixed_axis.png'
                # full_path_img7 = f'{mapper_graph_path}/mapper_graph_{generate_numerical_filename(initial_slice_position + i, len(data))}_{slice_size}.png'
                # full_path_mosaic = f'{mosaic_path}/mosaic_{generate_numerical_filename(initial_slice_position + i, len(data))}_{slice_size}.png'
                
                # generate_mosaic(full_path_img1, full_path_img2, full_path_img3, full_path_img4, full_path_img5, full_path_img6, full_path_img7, full_path_mosaic)                

            # Save the dataframe with the features
            if save:
                df_features.to_csv(f'{features_path}/features_{data_name}_time_delay_{time_delay_in_days}_slice_size_{slice_size}.csv', index=False)
                print(f'Features saved to {features_path}/features_{data_name}_time_delay_{time_delay_in_days}_slice_size_{slice_size}.csv')

            # df_features = pd.read_csv(f'{features_path}/features_{data_name}_time_delay_{time_delay_in_days}_slice_size_{slice_size}.csv')

            # list_columns_to_normalize = ['connected_components_entropy', 'loops_entropy', 'voids_entropy',
            #     'connected_components_amplitude', 'loops_amplitude', 'voids_amplitude',
            #     'connected_components_number_of_points', 'loops_number_of_points',
            #     'voids_number_of_points'
            #     ]
            # df_features_normalized = normalize_dataframe(df_features, list_columns_to_normalize)
            # df_features_normalized_v2 = normalize_dataframe(df_features, list_columns_to_normalize, (-1, 1))

            # for i in range(0, (len(data)-slice_size), num_point_for_1d):
            #     mosaic_path_img = f'{mosaic_path}/mosaic_{generate_numerical_filename(initial_slice_position + i, len(data))}_{slice_size}.png'
            #     features_path_img = f'{features_plot_path}/features_{generate_numerical_filename(initial_slice_position + i, len(data))}_{slice_size}.png'
            #     features_accumulate_path_img = f'{features_plot_path}/features_accumulate_{generate_numerical_filename(initial_slice_position + i, len(data))}_{slice_size}.png'
            #     mosaic_and_features_path_img = f'{mosaic_and_features_path}/mosaic_and_features_{generate_numerical_filename(initial_slice_position + i, len(data))}_{slice_size}.png'
            #     fig = generate_features_plot(data, column_name, df_features_normalized_v2, initial_slice_position + i, slice_size, save, features_plot_path, f'features_{generate_numerical_filename(initial_slice_position + i, len(data))}_{slice_size}')
            #     # fig = generate_features_plot(data, column_name, df_features_normalized_accumulated, initial_slice_position + i, slice_size, save, features_plot_path, f'features_accumulate_{generate_numerical_filename(initial_slice_position + i, len(data))}_{slice_size}')
            #     generate_mosaic_and_features(mosaic_path_img, features_path_img, mosaic_and_features_path_img)

            # create_mosaic_video(f'{mosaic_and_features_path}/', f'{video_path}/', f'mosaic_and_features_video_{slice_size}.mp4')

/var/folders/s0/_j22qz4n5j72grfwwy9xgbxc0000gn/T/ipykernel_13566/2204231434.py:77: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_features = pd.concat([df_features, pd.DataFrame({


Features saved to ../results/processed_log_return_wtmra_0_ADAUSDT_1d_time_delay_72_slice_size_336/features//features_ADAUSDT_1d_time_delay_72_slice_size_336.csv


/var/folders/s0/_j22qz4n5j72grfwwy9xgbxc0000gn/T/ipykernel_13566/2204231434.py:77: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_features = pd.concat([df_features, pd.DataFrame({


Features saved to ../results/processed_log_return_wtmra_0_ADAUSDT_1d_time_delay_168_slice_size_336/features//features_ADAUSDT_1d_time_delay_168_slice_size_336.csv


/var/folders/s0/_j22qz4n5j72grfwwy9xgbxc0000gn/T/ipykernel_13566/2204231434.py:77: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_features = pd.concat([df_features, pd.DataFrame({


...

Features saved to ../results/processed_log_return_wtmra_0_ADAUSDT_1d_time_delay_168_slice_size_720/features//features_ADAUSDT_1d_time_delay_168_slice_size_720.csv


/var/folders/s0/_j22qz4n5j72grfwwy9xgbxc0000gn/T/ipykernel_13566/2204231434.py:77: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_features = pd.concat([df_features, pd.DataFrame({


Features saved to ../results/processed_log_return_wtmra_0_ADAUSDT_1d_time_delay_72_slice_size_1080/features//features_ADAUSDT_1d_time_delay_72_slice_size_1080.csv


/var/folders/s0/_j22qz4n5j72grfwwy9xgbxc0000gn/T/ipykernel_13566/2204231434.py:77: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_features = pd.concat([df_features, pd.DataFrame({


Features saved to ../results/processed_log_return_wtmra_0_ADAUSDT_1d_time_delay_168_slice_size_1080/features//features_ADAUSDT_1d_time_delay_168_slice_size_1080.csv
